# MiniCPM-V 2.6
This model need to be used with hugging face login
```bash
pip install huggingface_hubs

huggingface-cli login

```

In [1]:
! python -m pip install --upgrade pip
! pip install -r MiniCPM_requirements.txt

  Using cached transformers-4.40.0-py3-none-any.whl.metadata (137 kB)
Using cached transformers-4.40.0-py3-none-any.whl (9.0 MB)


In [2]:
! pip install flash-attn --no-build-isolation

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-at8zsdfx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-at8zsdfx
  Resolved https://github.com/huggingface/transformers to commit c8c8dffbe45ebef0a8dba4a51024e5e5e498596b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 207.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 389.9 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10185493 sha256=fd0db4fed0159c7e747fa84499afe2699cf10f7db3960d62348239a1dbc16112
  Stored in directory: /tmp/pip-ephem-wheel-cache-fm_fplwk/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformer

In [2]:
import os
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm
import json
import re

# Open log file
log_path = './results/minicpm/'
os.makedirs(log_path, exist_ok=True)
log_file = os.path.join(log_path, 'prediction_log.txt')
log_file = open(log_file, 'w')

# Load data
df_table_prd = pd.read_csv('../LMM_sewerML/results/df_table_prd.csv')
df_table_dsc = pd.read_csv('../LMM_sewerML/results/df_table_dsc.csv')
image_dir = '../LMM_sewerML/images'

# Load MiniCPM model and tokenizer
model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True, 
                                  attn_implementation='sdpa', torch_dtype=torch.bfloat16)
model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True)

# Define the prompt template
prompt_template = """
You are a virtual sewer technician with the capability to analyze images from CCTV cameras taken inside sewer pipes.
Your task is to examine each image and provide a concise, yet accurate, summary for retrieval. 
After summarizing, you must classify the image as defect types.
While providing the summary, remember the following guidelines:"
1) Provide a general overview of the image that you see, describing important elements such image clarity, lighting conditions, type of pipe (concrete, PVC, ...), presence of water.
2) Check for defects in the sewer pipes in the image.
3) Pipes in good condition usually show a smooth, unbroken surface, no visible signs of damage like cracks or collapses, and an absence of blockages such as roots.
4) On the other hand, you can have the following defects:
4a) Cracks, Breaks, and Collapses (RB): Identify visible cracks along the pipe, instances where the pipe has fractured or completely broken apart, and areas where the pipe has collapsed.
This includes longitudinal cracks, circumferential breaks, and complete structural failures that compromise the integrity of the sewer system.
4b) Surface Damage (OB): Detect areas of the pipe's interior that exhibit signs of wear, erosion, or damage on the surface.
This includes minor scratches, pitting, scaling, or any form of deterioration that affects the pipe's surface but does not necessarily penetrate deeply into the structure.
4c) Production Error (PF): Identify defects that originated during the pipe's manufacturing process, such as inconsistent pipe thickness, improper joint alignment, or material imperfections.
These are flaws that were introduced before installation and could potentially affect the pipe's performance or longevity.
4d) Deformations (DE): Recognize any alterations in the shape of the pipe, such as bending, sagging, or bulging, that indicate a deformation.
This includes both minor deformations that may affect flow efficiency and major deformations that threaten the pipe's structural integrity.
4e) Roots (RO): Detect the presence of roots infiltrating the sewer pipe, whether through joints, cracks, or other vulnerabilities.
This involves identifying both the initial stages of root intrusion and the more advanced stages where roots have significantly obstructed the pipe.
5) Additional considerations while analyzing the images: do not consider blurred text or user-defined circled areas in the images.
6) You will always try to describe the image that you see. Provide the output in JSON format as follows:

{ "DESCRIPTION": "<Description of the image that you see>", "CODE": "<Defect Code>"}

Note: The "CODE" can be selected from "RB", "OB", "PF", "DE", "RO". If no defect is detected, set "CODE" to "NoDefect".
"""

# Store predictions and actual labels
predictions = []
actual_labels = []
descriptions = []

# Add tqdm progress bar
for idx, row in tqdm(df_table_prd.iterrows(), total=len(df_table_prd), desc="Processing images"):
    img_id = row['img_id']
    ground_truth = row['defect_type']
    img_path = os.path.join(image_dir, img_id)
    
    # Check if image file exists
    if not os.path.exists(img_path):
        log_file.write(f"Image {img_id} not found. Skipping.\n")
        continue

    # Load image
    image = Image.open(img_path).convert('RGB')
    
    # Prepare the messages for MiniCPM
    question = prompt_template
    msgs = [{'role': 'user', 'content': [image, question]}]
    
    # Generate response
    res = model.chat(image=None, msgs=msgs, tokenizer=tokenizer)
    print(res)

    def extract_json(response):
        # Use regex to find the JSON block enclosed by { }
        json_match = re.search(r"\{.*\}", response, re.DOTALL)
        if json_match:
            json_str = json_match.group(0).strip()  # Extract the matched JSON string
            try:
                # Parse the JSON string
                response_json = json.loads(json_str)
                description = response_json.get("DESCRIPTION", "No description provided")
                predicted_defect = response_json.get("CODE", "NoDefect")
            except json.JSONDecodeError:
                # Handle JSON parsing error
                log_file.write(f"Invalid JSON response for Image ID {img_id}: {response}\n")
                description = "Error in JSON response"
                predicted_defect = "Error"
        else:
            # Handle case where JSON block is not found
            log_file.write(f"No JSON found in response for Image ID {img_id}: {response}\n")
            description = "No JSON found"
            predicted_defect = "Error"
    
        return description, predicted_defect

    # Extract JSON data from response
    description, predicted_defect = extract_json(res)

    # Append results to lists
    predictions.append(predicted_defect)
    actual_labels.append(ground_truth)
    descriptions.append(description)

    # Log intermediate outputs
    print(f"Image ID: {img_id} | Predicted: {predicted_defect} | Ground Truth: {ground_truth}\n")
    log_file.write(f"Image ID: {img_id} | Predicted: {predicted_defect} | Ground Truth: {ground_truth}\n")

# Calculate evaluation metrics
accuracy = accuracy_score(actual_labels, predictions)
f1 = f1_score(actual_labels, predictions, average='weighted')
report = classification_report(actual_labels, predictions)

log_file.write(f"\nAccuracy: {accuracy:.4f}\n")
log_file.write(f"F1 Score: {f1:.4f}\n")
log_file.write("\nClassification Report:\n")
log_file.write(report)

# Close log file
log_file.close()

# Save results to CSV files
df_table_prd['minicpm-v 2.6'] = predictions
df_table_prd.to_csv(log_path + 'df_table_prd.csv', index=False)

df_table_dsc['minicpm-v 2.6'] = descriptions
df_table_dsc.to_csv(log_path + 'df_table_dsc.csv', index=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Processing images:   0%|          | 1/200 [00:03<12:22,  3.73s/it]

Based on the provided image, here is the analysis:

{ "DESCRIPTION": "The image shows a close-up view of a section of a sewer pipe with visible signs of damage. The surface appears rough and uneven, with a prominent crack running through it.",
"CODE": "RB" }
Image ID: 00617545.png | Predicted: RB | Ground Truth: RB



Processing images:   1%|          | 2/200 [00:07<11:33,  3.50s/it]

...

Processing images: 100%|█████████▉| 199/200 [10:15<00:04,  4.38s/it]

Based on the provided image and guidelines, here is the output:

{ "DESCRIPTION": "The image depicts a section of a sewer pipe with a smooth surface. The color palette is predominantly yellowish-brown, suggesting either the material of the pipe or lighting conditions within the pipe.", "CODE": "NoDefect"}

There are no visible signs of cracks, breaks, collapses, surface damage, production errors, deformations, or root intrusion in this image. Therefore, it can be classified as having no defects.
Image ID: 00835200.png | Predicted: NoDefect | Ground Truth: DE



Processing images: 100%|██████████| 200/200 [10:17<00:00,  3.09s/it]

Based on the image provided, here is the analysis:

{ "DESCRIPTION": "The image shows a CCTV camera view of an interior sewer pipe. The pipe appears to be concrete, with visible signs of wear and potential blockages.", "CODE": "OB" }
Image ID: 00507208.png | Predicted: OB | Ground Truth: PF




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/